In [ ]:
# Testing params
excel_path=r"C:\Users\lwoods\Documents\LW_Projects_folder\01_MO\G02_Barbacid\R12_Oksana\P01_KRS1\E03_1gel_2_isoforms\CNIO_prot_core_results\MaxQuant_protein_groups_results.xlsx"
fasta_file=r"C:\Users\lwoods\Documents\LW_Projects_folder\01_MO\G02_Barbacid\R12_Oksana\P01_KRS1\E03_1gel_2_isoforms\fasta_folder\merged_fasta_folder\custom_fasta_merged_w_HUMAN_database.fasta"
POIs=['P23246', 'Q9BUJ2', 'P53992', 'Q00839', 'Q8IVT5', 'A0A2R8Y4X0', 'A0A2R8Y5H9']
experiments=['Down','Up']
identification="Razor + Unique"
colours=['#FF5900', '#DA16FF', '#1CA71C', '#2E91E5', '#E15F99', '#25B0B0', '#B68100', '#750D86', '#EB663B', '#511CFB', '#00A08B', '#FB00D1', '#FC0080', '#B2828D', '#6C7C32', '#778AAE', '#862A16', '#A777F1', '#620042', '#1616A7', '#DA60CA', '#6C4516', '#0D2A63', '#AF0038']
reattribute_peptides=False

In [ ]:
from nbconvert import HTMLExporter
from nbformat import read, write
from pathlib import Path
import os
from IPython.display import Markdown, display
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
from Bio import SeqIO
import plotly.graph_objects as go
import re
from matplotlib.colors import to_rgb

# Report text stuff. Looks a bit dumb right now
# so TODO: adjust text parameters, etc. to make
# a nicer report
# Also, all figures should be output as standalone htmls
# so need to include output folder as parameter

def printmd(string):
    from IPython.display import Markdown, display
    markdown_string=display(Markdown(string))


def format_tick(n: float):
    m, e = f"{n:.0e}".split("e")
    e = int(e)
    return f"{m} x 10<sup>{e}</sup>"


def extract_protein_names(row, prot_id_col, POIs):
    headers = row['Fasta headers'].split(';') if not pd.isna(row['Fasta headers']) else []
    majority_ids = row[prot_id_col].split(';')

    names = []
    for header, majority_id in zip(headers, majority_ids):
        if majority_id in POIs:
            names.append(majority_id)
        else:
            parts = header.split('|')
            if len(parts) >= 3:
                try:
                    identifier, description = parts[2].split(" ", 1)
                    names.append(identifier)
                except:
                    names.append(majority_id)
            else:
                names.append(majority_id)

    names.extend(majority_ids[len(headers):])

    return ';'.join(names)

# Get df of POIs but make new col for names for plotting
# (possibly more trouble than it's worth....)
def get_POIs_df(experiment, protein_groups, num, POIs):
    protein_groups['Plotting names'] = protein_groups.apply(extract_protein_names, args=(prot_id_col, POIs), axis=1)
    mask = protein_groups['Plotting names'].str.split(';').apply(lambda x: any(poi in x for poi in POIs))
    POIs_df = protein_groups[mask]
    return POIs_df

# Get the top n proteins (after subtracting POIs)
# n is buried right now, should be more accessible
# and also may be better to use intensity thresh instead
def protein_df_organization(experiment,protein_groups,num,POIs_df,prot_id_col):
    num_POIs=len(POIs_df)
    intensity_col='Intensity '+str(experiment)
    head_n=num-num_POIs
    protein_groups['Plotting names'] = protein_groups.apply(extract_protein_names, args=(prot_id_col,POIs), axis=1)
    total_intensity=protein_groups[intensity_col].sum()
    top_n_intensity=protein_groups[~protein_groups[prot_id_col].isin(POIs_df[prot_id_col])].sort_values(intensity_col, ascending=False).head(head_n)
    top_n=pd.concat([POIs_df, top_n_intensity], axis=0)
    top_n['Relative protein intensity '+str(experiment)+' [%]']=top_n[intensity_col]/total_intensity*100
    return top_n

def make_report_title(experiment,POIs,identification):
    POI_string1=(', ').join(POIs[0:-2])
    POI_string2=(' and ').join(POIs[-2:])
    if len(POIs)>2:
        POI_string_full=(', ').join([POI_string1,POI_string2])
    else:
         POI_string_full=POI_string2
    printmd(f'<h1 style="font-family:Open Sans, verdana, arial, sans-serif;font-weight:normal;">Report for MS analysis of {experiment}: {POI_string_full}</h1>')
    if identification=="Unique":
        printmd(f'<span style="font-family:Open Sans, verdana, arial, sans-serif;font-size:12;margin-top:0px;margin-bottom:0px;">Note that this report only considers peptides unique to the protein of interest.<br><br><br></span>')

def make_subtitle(num,POIs,POIs_df):
    num_POIs=len(POIs_df)
    new_POIs=POIs_df['Plotting names'].values.tolist()
    POI_string1=(', ').join(new_POIs[0:-2])
    POI_string2=(' and ').join(new_POIs[-2:])
    if len(new_POIs)>2:
        POI_string_full=(', ').join([POI_string1,POI_string2])
    else:
        POI_string_full=POI_string2
    unidentified_POIs=[]
    for POI in POIs:
        is_present = POIs_df['Plotting names'].apply(lambda x: POI in x.split(';')).any()
        if is_present == False:
            unidentified_POIs.append(POI)
    if len(unidentified_POIs)<len(POIs):
        printmd(f'<h2 style="font-family:Open Sans, verdana, arial, sans-serif;font-weight:normal;margin-top:0px;margin-bottom:0px;">Protein signal intensities</h2><span style="font-family:Open Sans, verdana, arial, sans-serif;font-size:12;margin-top:0px;margin-bottom:0px;">The following displays the raw signal intensities corresponding to {POI_string_full} with the top {num-num_POIs} proteins.<span>')
    else:
        printmd(f'<h2 style="font-family:Open Sans, verdana, arial, sans-serif;font-weight:normal;margin-top:0px;margin-bottom:0px;">Protein signal intensities</h2><span style="font-family:Open Sans, verdana, arial, sans-serif;font-size:12;margin-top:0px;margin-bottom:0px;">The following displays the raw signal intensities corresponding to the top {num-num_POIs} proteins.<span>')
    if len(unidentified_POIs)>0:
        unid_POIs_string=', '.join(unidentified_POIs)
        printmd(f'<span style="font-family:Open Sans, verdana, arial, sans-serif;font-size:12;margin-top:0px;margin-bottom:0px;">The following proteins of interest were not identified in any experiment: {unid_POIs_string}.<span>')

def shorten_name(name, max_len=26):
    return name[:max_len-3] + '...' if len(name) > max_len else name        

def topn_proteins_identified(top_n,POIs,POIs_df,num,experiment,colours,prot_id_col):
    leg_names=top_n['Plotting names'].values.tolist()
    top_n['Plotting names'] = top_n['Plotting names'].apply(lambda x: shorten_name(x))
    
    group_colour_mapping = {}
    for poi, color in zip(sorted(POIs), colours):
        group_colour_mapping[poi] = color

    for majority_prot_list in top_n['Plotting names']:
        proteins = majority_prot_list.split(";")
        if any("CON__" in protein for protein in proteins):
            group_colour_mapping[majority_prot_list] = '#a6a3a2'
        elif majority_prot_list not in POIs:
            if any(poi in proteins for poi in POIs):
                for poi in POIs:
                    if poi in proteins:
                        group_colour_mapping[majority_prot_list] = group_colour_mapping[poi]
                        break
            else:
                group_colour_mapping[majority_prot_list] = 'black'
    
    intensity_col='Intensity '+str(experiment)
    num_POIs=len(POIs_df)
    POIs_plotting=POIs_df['Plotting names'].values.tolist()

    def format_text(value):
        float_value = float(value)
        if float_value == 0.0:
            return "Absent"
        elif round(float_value, 2) == 0:
            return "<0.01%"
        else:
            return f'{round(float_value, 2)}%'

    text_labels = [format_text(val) for val in top_n['Relative protein intensity '+str(experiment)+' [%]']]
    
    columns_to_fill = [prot_id_col, 'Gene names', 'Protein names', 'Potential contaminant', 'MS/MS count']     
    top_n['Potential contaminant'] = top_n['Potential contaminant'].map({'+': 'Yes', '': 'No'}).fillna('No')

    # Handling protein groups
    def extract_gene_names(header):
        if ";" in str(header):
            headers = str(header).split(";")
        else:
            headers = [str(header)]
        gene_names = [re.search(r'GN=([\w\d]+)', h).group(1) if re.search(r'GN=([\w\d]+)', h) else '' for h in headers]
        return ";".join(gene_names)

    def extract_protein_names(header):
        if ";" in str(header):
            headers = str(header).split(";")
        else:
            headers = [str(header)]
        protein_names = [re.search(r'\|([^|]+)\s+OS=', h).group(1) if re.search(r'\|([^|]+)\s+OS=', h) else '' for h in headers]
        return ";".join(protein_names)
    
    if 'Gene names' not in top_n.columns:
        top_n['Gene names'] = top_n['Fasta headers'].apply(extract_gene_names)

    if 'Protein names' not in top_n.columns:
        top_n['Protein names'] = top_n['Fasta headers'].apply(extract_protein_names)
    
    for col in columns_to_fill:
        top_n[col].fillna("", inplace=True)

    fig = px.bar(top_n, 
                 x=top_n['Plotting names'], 
                 y=top_n[intensity_col], 
                 color=top_n['Plotting names'], 
                 color_discrete_map=group_colour_mapping, 
                 text=text_labels,
                 hover_data={'Plotting names': False, intensity_col: True},
                 custom_data=[
                              prot_id_col, 'Gene names', 'Protein names','Potential contaminant',
                              'MS/MS count',
                             ])
    # Possible TODO: make the hoverables inputs, but may be a bit too much
    hovertemplate = (
        "<b>Protein/Protein group</b>: %{x}<br>"
        "<b>Intensity</b>: %{y}<br>"
        "<b>Prot ID</b>: %{customdata[0]}<br>"
        "<b>Gene Names</b>: %{customdata[1]}<br>"
        "<b>Protein Names</b>: %{customdata[2]}<br>"
        "<b>Potential contaminant</b>: %{customdata[3]}<br>"
        "<b>MS/MS count</b>: %{customdata[4]}<br>"
    )

    fig.update_traces(texttemplate='%{text}', 
                      textposition='outside', 
                      hovertemplate=hovertemplate)
    
    min_value = top_n[intensity_col].min()
    max_value = top_n[intensity_col].max()
    y_axis_limit = max_value + 0.10 * max_value

    fig.update_layout(
        title=f'Signal intensity (peptide sum) of top {num} proteins identified<br><sup>Bars are marked with relative protein intensity. Known contaminants are marked in grey and other proteins in black.<sup>',
        yaxis_title="Intensity", 
        xaxis_title="Protein group",
        plot_bgcolor='#EEEAE8',
        legend_title_text='Proteins',
        yaxis={'tickformat':"e", 'range': [0, y_axis_limit], 'ticks': 'outside'},
        font=dict(color='black',size=12)
    )

    config = {
      'toImageButtonOptions': {
        'format': 'png',
        'filename': 'Protein_report_plot',
        'scale': 4
      }
    }
    
    fig.show(renderer="notebook", config=config)

    
def filter_peptides(peptides_df, protein_groups_df, quantification,prot_id_col,pept_id_col):
    final_peptides_df = pd.DataFrame()

    # Iterating (:/)--can't think of a better way round rn
    for _, row in protein_groups_df.iterrows():
        protein_group = row[prot_id_col]
        if quantification == "All":
            filtered_peptides = peptides_df[peptides_df[pept_id_col].str.contains(protein_group)]
        elif quantification == "Unique":
            filtered_peptides = peptides_df[peptides_df[pept_id_col].str.contains(protein_group) & (peptides_df['Unique (Groups)'] == 'yes')]
        elif quantification == "Razor + Unique":
            # Now need to grab the "is razor" list to sort through so I can properly determine
            # which peptides to show depending on quant mode
            #razor_status = row['Peptide is razor'].split(";")
            #peptide_ids = row['Peptide IDs'].split(";")
            peptide_is_razor = [x == 'True' for x in row['Peptide is razor'].split(';')]
            peptide_ids = list(map(int, row['Peptide IDs'].split(';')))
            current_protein_peptides = peptides_df[peptides_df['id'].isin(peptide_ids)].copy()
            current_protein_peptides['is_razor'] = current_protein_peptides['id'].map(dict(zip(peptide_ids, peptide_is_razor)))
            filtered_peptides = current_protein_peptides[(current_protein_peptides['is_razor']) | 
                                (current_protein_peptides['Unique (Groups)'] == 'yes')]
            
        # Accumulate dfs
        final_peptides_df = pd.concat([final_peptides_df, filtered_peptides])

    return final_peptides_df

    
def peptide_plots(fasta_file,peptides_df,POIs,protein_df,exp,colours,prot_id_col,pept_id_col):
    def read_fasta(fasta_file):
        # Assumes uniprot--should make adjustable
        fasta_dict ={}
        for record in SeqIO.parse(fasta_file, "fasta"):
            protein_id = record.id.split("|")[1]  
            fasta_dict[protein_id]=[str(record.seq),len(record.seq)]
        return fasta_dict

    def filter_peptides(peptides_df, exp):
        df = peptides_df[peptides_df[f'Intensity {exp}'] != 0]
        return df

    def get_protein_peptides(peptide_df, protein_df, exp, prot_id_col, pept_id_col):
        protein_peptide_dict = {}
        # Loop through protein groups
        for _, row in protein_df.iterrows():
            proteins = row[prot_id_col].split(';')
            peptide_ids = row['Peptide IDs'].split(';')
            for protein in proteins:
                valid_peptides = []
                for peptide_id in peptide_ids:
                    peptide_row = peptide_df[peptide_df['id'] == int(peptide_id)]
                    if peptide_row.empty:
                        continue
                    if protein in peptide_row[pept_id_col].iloc[0].split(';'):
                        peptide_sequence = peptide_row['Sequence'].iloc[0]
                        intensity = peptide_row[f'Intensity {exp}'].iloc[0]
                        # Find start and end of peptide within protein sequence
                        # Remember to correct for Python indexing!
                        start = seq_dict[protein][0].find(peptide_sequence) + 1 
                        end = start + len(peptide_sequence) - 1
                        if start > 0: 
                            valid_peptides.append((start, end, intensity, peptide_sequence))
                protein_peptide_dict[protein] = valid_peptides
        return protein_peptide_dict

    def combine_ranges(ranges):
        # This is for "gluing" overlapping and touching peptides
        if not ranges: 
            return []

        ranges = sorted(ranges, key=lambda x: x[0])  
        combined = [(ranges[0][0], ranges[0][1])] 

        for start, end, intensity, sequence in ranges[1:]:
            last_start, last_end = combined[-1]
            if start <= last_end:
                combined[-1] = (last_start, max(last_end, end))
            else:
                combined.append((start, end))

        return combined


    def calculate_coverage_percentage(combined_ranges, seq_length):
        coverage = sum(end - start for start, end in combined_ranges)
        percentage_coverage = (coverage / seq_length) * 100
        return percentage_coverage
    
    def calculate_brightness(color):
        # Convert hex to RGB
        r, g, b = to_rgb(color)
        # 1-255
        r, g, b = [int(x * 255) for x in [r, g, b]]
        # equation for finding brightness
        brightness = (0.299 * r) + (0.587 * g) + (0.114 * b)

        return brightness

    def generate_html_representation(protein_sequence, coverage_ranges, color="red"):
        marked_sequence = ""
        position = 0

        # Calculate brightness and decide text color
        brightness = calculate_brightness(color)
        text_color = "black" if brightness > 128 else "white"

        for start, end in coverage_ranges:
            start = int(start) - 1  
            marked_sequence += protein_sequence[position:start]  
            marked_sequence += f'<span style="background-color:{color}; color:{text_color}">{protein_sequence[int(start):int(end)]}</span>'  # Covered region
            position = int(end)  

        marked_sequence += protein_sequence[position:] 
        return marked_sequence

    # Run dependecy functions
    prot_colour_mapping={}
    #print(sorted(POIs_df[prot_id_col].values.tolist()))
    #print(colours)
    for majority_prot_list,c in zip(sorted(POIs),colours):
        for prot in majority_prot_list.split(";"):
            prot_colour_mapping[prot]=c

    seq_dict = read_fasta(fasta_file)
    peptide_df = filter_peptides(peptides_df, exp)
    protein_peptide_dict = get_protein_peptides(peptide_df, POIs_df, exp, prot_id_col, pept_id_col)
    html_proteins={}

    # Plot coverage
    for protein, peptides in protein_peptide_dict.items():
        if not peptides:
            #printmd(f'<span style="font-family:Open Sans, verdana, arial, sans-serif;font-size:12;margin-top:0px;margin-bottom:0px;"No valid peptides for protein: {protein}"<span>')
            continue
        combined = combine_ranges(protein_peptide_dict[protein])
        seq_length=seq_dict[protein][1]
        seq_string=seq_dict[protein][0]
        seq_cov=calculate_coverage_percentage(combined, seq_length)

        fig = go.Figure()
        for s, e in combined:
            fig.add_trace(
                go.Scatter(x=[s-0.5,s-0.5,e+0.5,e+0.5,s-0.5],y=[0,1,1,0,0],fill="toself",
                fillcolor = prot_colour_mapping[protein],
                mode='none',line=dict(width=0),showlegend=False,hoverinfo='skip')
            )
        fig.add_trace(go.Scatter(x=[0.5,0.5,seq_length+0.5,seq_length+0.5,0.5],y=[0,1,1,0,0],
                                 line=dict(color='black',width=1.5),
                                mode='lines',showlegend=False,hoverinfo='skip')
                     )
        fig.update_xaxes(range=[0.5, seq_length+1],showgrid=False)
        fig.update_yaxes(range=[0, 1],showgrid=False,showticklabels=False,ticklen=0)
        fig.update_layout(
            title=f'Peptidyl signal coverage for {protein}<br><sup>Sequence coverage: {round(seq_cov,2)}%<sup>', 
            xaxis_title="Amino acid position",
            plot_bgcolor='#EEEAE8',
            #legend_title_text='Proteins',
            yaxis={'tickformat':".2", 'rangemode': 'tozero',
                   'ticks': 'outside'},
            font=dict(color='black',size=12),
            autosize=False,
            width=800,
            height=200,
        )
        
        config = {
          'toImageButtonOptions': {
            'format': 'png', # one of png, svg, jpeg, webp
            'filename': 'Peptide_coverage_plot',
            'scale': 4 # Multiply title/legend/axis/canvas sizes by this factor
          }
        }
        # Show coverage plot
        fig.show(config=config)

        # Text coverage
        #html_proteins[protein] = generate_html_representation(seq_string, combined, prot_colour_mapping[protein])
        html_representation = generate_html_representation(
            seq_string,
            combined, 
            color=prot_colour_mapping[protein]
        )
        printmd(
        '<div style="width:80ch;font-size:14px;font-family:Open Sans, verdana, arial, sans-serif;margin-left:3em;">'
        '<p style="word-break: break-all">'
        +html_representation+
        '</p>'
        '</div>'
        )
        
        # Peptide intensity plot
        fig2 = go.Figure()
        
        intensity_values=[]
        
        for s, e, inten, pep_sequence in peptides:
            if inten > 0:
                intensity_values.append(inten)
                fig2.add_trace(go.Scatter(x=[s,e],y=[inten,inten],
                                         #fill="toself",
                        mode='lines',line=dict(color=prot_colour_mapping[protein]),
                                         #line=dict(width=100)
                                         showlegend=False,
                                         hovertemplate =
                                        f'Start position: {s}'+
                                        f'<br>End position: {e}'+
                                        f'<br>Intensity: {inten:e}'+
                                        f'<br>Sequence: {pep_sequence}'
                                        )
                         )
                middle_x = (s + e) / 2
                fig2.add_trace(
                            go.Scatter(x=[middle_x], y=[inten], 
                            mode='markers', marker=dict(size=0, opacity=0, color=prot_colour_mapping[protein]),
                            showlegend=False,
                            hovertemplate=
                                       f'Start position: {s}'+
                                       f'<br>End position: {e}'+
                                       f'<br>Intensity: {inten:e}'+
                                       f'<br>Sequence: {pep_sequence}'
                                      )
                            )
        
        fig2.update_traces(name='')
        
        max_val = np.max(intensity_values)
        min_val = np.min(intensity_values)
        # For adjusting plots y-axis
        
         # Calculate log base 10 of min and max
        log_min_val = np.floor(np.log10(min_val)) if min_val > 0 else 0
        log_max_val = np.ceil(np.log10(max_val))

        # Want to ensure that there's more than one major grid line
        if (log_min_val == log_max_val) or (log_max_val - log_min_val <= 2):
            log_max_val += 1  # one order of magnitude larger

        # get tick values
        tick_vals = [10 ** i for i in range(int(log_min_val), int(log_max_val) + 1)]
        tick_text = [f"{x:.0e}" for x in tick_vals]
        
        fig2.update_layout(
            title=f'Peptide signal intensity for {protein}',
            yaxis_title=f'Peptide intensity', 
            xaxis_title="Amino acid position",
            xaxis=dict(
            range=[1, seq_length], 
            ),
            plot_bgcolor='#EEEAE8',
            font=dict(color='black',size=12),
            yaxis=dict(
            type="log",
            tickvals=tick_vals,
            ticktext=tick_text,
            range=[np.log10(min(tick_vals)), np.log10(max(tick_vals))]  # Log scale range
            )
          )
        config = {
          'toImageButtonOptions': {
            'format': 'png',
            'filename': 'Peptide_intensity_plot',
            'scale': 4
          }
        }
        fig2.show(config=config)

In [ ]:
# Number of proteins to consider
num=10
protein_groups = pd.read_excel(excel_path,sheet_name="Full_proteinGroups")
peptides_data = pd.read_excel(excel_path,sheet_name="Peptides") 
    
if reattribute_peptides==False:
    prot_id_col='Majority protein IDs'
    pept_id_col='Proteins'
elif reattribute_peptides=="no":
    prot_id_col='Majority protein IDs'
    pept_id_col='Proteins'
elif reattribute_peptides==True:
    prot_id_col='Reattributed Protein'
    pept_id_col='Reattributed Protein'
elif reattribute_peptides=="yes":
    prot_id_col='Reattributed Protein'
    pept_id_col='Reattributed Protein'
      
for experiment in sorted(experiments):
    POIs_df=get_POIs_df(experiment,protein_groups,num,POIs)
    #updated_POIs=return_new_POIs(POIs,POIs_df)
    top_n=protein_df_organization(experiment,protein_groups,num,POIs_df,prot_id_col)
    make_report_title(experiment,POIs,identification)
    make_subtitle(num,POIs,POIs_df)
    topn_proteins_identified(top_n,POIs,POIs_df,num,experiment,colours,prot_id_col)
    try:
        relevant_peptides_df = filter_peptides(peptides_data, POIs_df, identification,prot_id_col,pept_id_col)
        peptide_plots(fasta_file,relevant_peptides_df,POIs,POIs_df,experiment,colours,prot_id_col,pept_id_col)
    except KeyError:
        pass